## SFINCS - grpc4bmi example

This notebook shows how the SFINCS BMI server built in this package can be used
with grpc4bmi. It assumes you have built the docker container with

```
cd src
docker build -t sfincs-bmiserver .
```

### Requirements

We use grpc4bmi to communicate with the BMI model inside a container.

In [1]:
# pip installa grpc4bmi

### Case specification

SFINCS runs from within the working directory where the input files are located.
CD to the humber case dir

In [2]:
%cd /home/peter/ewatercycle/sfincs-bmi-server/cases/humber/sfincs_humber_executed
%ls

/home/peter/ewatercycle/sfincs-bmi-server/cases/humber/sfincs_humber_executed
figs/        log.txt     sfincs.bzs  sfincs.ind  sfincs.msk  sfincs.src
gis/         precip.nc   sfincs.dep  sfincs.inp  sfincs.obs  sfincs_his.nc
hydromt.log  sfincs.bnd  sfincs.dis  sfincs.man  sfincs.scs  sfincs_map.nc


Lets use the example parameter set in https://github.com/Deltares/hydromt_sfincs/tree/main/examples/sfincs_compound

In [1]:
# !git clone https://github.com/Deltares/hydromt_sfincs/
# from pathlib import Path
# parameter_set_dir = Path("hydromt_sfincs/examples/sfincs_compound").absolute()

fatal: destination path 'hydromt_sfincs' already exists and is not an empty directory.


Example above returns 0 for each time method, which is incorrect so we will use the alternate parameter set below

In [ ]:
!cp -r ~/dcache/parameter-sets/sfincs_humber ./


In [1]:
from pathlib import Path
parameter_set_dir = Path("sfincs_humber").absolute()
parameter_set_dir

PosixPath('/home/verhoes/git/eWaterCycle/sfincs-bmi-server/sfincs_humber')

### Instantiate the model

This starts the model container and connects the client to it.
We can already get the component name

In [12]:
from grpc4bmi.bmi_client_docker import BmiClientDocker
import numpy as np

In [27]:
model = BmiClientDocker(image='sfincs-bmiserver', image_port=50051, work_dir=str(parameter_set_dir), delay=1)


Sometime stdout/stderr is swallowed start server manually

```
docker run -ti --rm \
-v /home/verhoes/git/eWaterCycle/sfincs-bmi-server/sfincs_humber:/home/verhoes/git/eWaterCycle/sfincs-bmi-server/sfincs_humber \
-w /home/verhoes/git/eWaterCycle/sfincs-bmi-server/sfincs_humber -p 50051:50051 \
--entrypoint /bin/bash \
sfincs-bmiserver
# inside container
/usr/local/bin/sfincs_bmi_server
```

and connect with

```
from grpc4bmi.bmi_grpc_client import BmiClient
import grpc
model = BmiClient(grpc.insecure_channel("localhost:50051"))
```


In [21]:
model.get_component_name()

'Sfincs hydrodynamic model (C)'

### Interact with the model

We can initialize the model and see that it had effect: time variables seem to be set from the input file

In [28]:
model.initialize(str(parameter_set_dir / 'sfincs.inp'))

In [29]:
model.update()

In [7]:
{
    'get_current_time': model.get_current_time(), 
    'get_start_time':model.get_start_time(),
    'get_end_time': model.get_end_time(), 
    'get_time_units': model.get_time_units(), 
    'get_time_step': model.get_time_step(),
}

{'get_current_time': 950400.0,
 'get_start_time': 950400.0,
 'get_end_time': 950400.0,
 'get_time_units': 's',
 'get_time_step': 9.999999974752427e-07}

In [8]:
# Var methods
{
    'get_input_var_names': model.get_input_var_names(),
    'get_output_var_names': model.get_output_var_names(),
    'get_var_grid': model.get_var_grid('zs'),
    'get_var_itemsize': model.get_var_itemsize('zs'),
    'get_var_nbytes': model.get_var_nbytes('zs'),
    'get_var_type': model.get_var_type('zs'),
    'get_var_units': model.get_var_units('zs'),
}

{'get_input_var_names': ('zs', 'zb', 'qtsrc', 'zst_bnd'),
 'get_output_var_names': ('z_xz', 'z_yz', 'zs', 'zb', 'qtsrc', 'zst_bnd'),
 'get_var_grid': 0,
 'get_var_itemsize': 4,
 'get_var_nbytes': 1254036,
 'get_var_type': 'float',
 'get_var_units': 'm above reference level'}

In [129]:
# model.get_var_location('zs')
# Throws grpc4bmi erorr, maybe because enum in C and Python not matching

In [9]:
# Grid methods
grid_id = 0
{
    'get_grid_rank': model.get_grid_rank(grid_id),
    'get_grid_size': model.get_grid_size(grid_id),
    'get_grid_type': model.get_grid_type(grid_id),
}


{'get_grid_rank': 2, 'get_grid_size': 313509, 'get_grid_type': 'rectilinear'}

In [30]:
shape = model.get_grid_shape(grid_id, np.empty(model.get_grid_rank(grid_id), dtype=int))
shape

array([313509,      7])

In [11]:
# x = model.get_grid_x(grid_id, np.empty(shape[1]))
# x
# Gives segmentation fault which kills container, probably because the size of the array is not correct

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:37329 {created_time:"2023-10-31T14:13:25.167536345+01:00", grpc_status:14, grpc_message:"Socket closed"}"
>

In [18]:
# water level
model.get_value('zs', np.zeros(shape[0]))

array([-8.96797761e+14,  4.57594014e-41,  6.32876000e+05, ...,
        7.07476000e+05,  7.07576000e+05,  7.07676000e+05])

Updating the model should advance the model time. However, the initial timestep
is very small! The get_current_time value doesn't even seem to render to such
precision. This way it will take a million update steps just to advance the
model 1 time unit.

In [26]:
for i in range(10):
    print(model.get_current_time(), model.get_time_step())
    model.update()
    

950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07
950400.0 9.999999974752427e-07


Let's try getting it past the first 10 time units...

In [8]:
t = model.get_current_time()
while t < model.get_start_time() + 10:
    model.update()
    t = model.get_current_time()

print(t)

Exception ignored in: <function BmiClientDocker.__del__ at 0x7fef2406ef80>
Traceback (most recent call last):
  File "/home/verhoes/mambaforge/envs/ewatercycle/lib/python3.10/site-packages/grpc4bmi/bmi_client_docker.py", line 105, in __del__
    self.container.stop()
AttributeError: 'BmiClientDocker' object has no attribute 'container'


KeyboardInterrupt: 

In [27]:
print(model.get_current_time(), model.get_time_step())

950400.0 9.999999974752427e-07


After 10 minutes, nothing seems to have happened!

When I enforced the timestep to be 1 instead of `model.get_time_step()` (in the
c++ wrapper), the model actually did advance and the time step value also
changed to a more logical value (order of 1).

In [28]:
print(model.container.logs().decode())

BMI grpc server attached to server address 0.0.0.0:50051
 BMI init start
 config file: /home/verhoes/git/eWaterCycle/sfincs-bmi-server/sfincs_humber/sfincs.inp

 ----------- Welcome to SFINCS -----------

  @@@@@  @@@@@@@ @@ @@  @@   @@@@   @@@@@ 
 @@@ @@@ @@@@@@@ @@ @@@ @@ @@@@@@@ @@@ @@@
 @@@     @@      @@ @@@ @@ @@   @@ @@@    
  @@@@@  @@@@@@  @@ @@@@@@ @@       @@@@@ 
     @@@ @@      @@ @@ @@@ @@   @@     @@@
 @@@ @@@ @@      @@ @@  @@  @@@@@@ @@@ @@@
  @@@@@  @@      @@ @@   @   @@@@   @@@@@ 

              ..............              
          ......:@@@@@@@@:......          
       ..::::..@@........@@.:::::..       
     ..:::::..@@..::..::..@@.::::::..     
    .::::::..@@............@@.:::::::.    
   .::::::..@@..............@@.:::::::.   
  .::::::::..@@............@@..::::::::.  
 .:::::::::...@@.@..@@..@.@@..::::::::::. 
 .:::::::::...:@@@..@@..@@@:..:::::::::.. 
 ............@@.@@..@@..@@.@@............ 
 ^^^~~^^~~^^@@..............@@^^^~^^^~~^^ 
 .::::::::::@@......

In [16]:
# model.container.stop()

In [25]:
model.set_value('zs', np.zeros(shape[0]))


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:49089 {grpc_message:"Socket closed", grpc_status:14, created_time:"2023-10-31T14:16:07.537850043+01:00"}"
>

In [31]:

model.get_value('zs', np.zeros(shape[0]))

array([1.65429988e+34, 4.57355793e-41, 0.00000000e+00, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])